# Generating Session Summary with LLMs

This project utilizes LlamaIndex and AI technology to analyze Formula 1 car data and generate session summaries. It is particularly beneficial for race engineers seeking detailed insights and performance analysis during races.


In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Optional: Setup Observability

Here we will use our Arize Phoenix integration to view traces through the query engine. It will be available at http://localhost:6006


In [18]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"  # Phoenix receiver address

tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

WARNI [opentelemetry.instrumentation.instrumentor] Attempting to instrument while already instrumented


### Start SQL Database


In [4]:
from sqlalchemy import create_engine
from llama_index.core import SQLDatabase

engine = create_engine('sqlite:///spain_practice_1.db')

sql_database = SQLDatabase(engine, include_tables=["mercedes"])

### Define the LLM and the Embedding


In [19]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

### Define the Retriever and the Query Engine


In [7]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["mercedes"], return_raw=True
)

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

### Start retrieving information to construct the report


In [8]:
response = query_engine.query(
    "Which driver was faster on average on sector 1?"
)

I0000 00:00:1723244252.910416 2380678 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [9]:
print(">>> ", str(response))

>>>  RUS was faster on average on sector 1.


In [15]:
response2 = query_engine.query(
    "Which driver was faster on average on sector 2?"
)

print(">>> ", str(response2))

>>>  HAM


In [20]:
response3 = query_engine.query(
    "Write a summary about the driver HAM"
)

print(">>> ", str(response3))

>>>  The driver HAM is 28 years old and has a lap time of 98.0705652173913 seconds with a top speed of 289.0 km/h.
